In [2]:
%load_ext autoreload
%autoreload 2
#Assessments with confirmed active results
#107617, 107614
from modules.auth import *
from modules.assessments_endpoints import *
from modules.config import base_url_illuminate
import logging
import os

os.makedirs('logs', exist_ok=True)
logging.basicConfig(filename='./logs/Illuminate_operations.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', force=True)
logging.info('\n\n-------------New Illumiante Operations Logging Instance')

access_token, expires_in = get_access_token()
#Provide a frame of all assessments and all info on them
results = get_all_assessments(access_token) 
#get entire list of assessment ids
assessment_id_list = list(results['assessment_id'].unique())

test_results_standard, log_results_standard = loop_through_assessment_scores(access_token, assessment_id_list, 'Standard')
test_results_no_standard, log_results_no_standard = loop_through_assessment_scores(access_token, assessment_id_list, 'No_Standard')


temp = pd.concat([test_results_no_standard, test_results_standard], axis=0)
pd.set_option('display.max_columns', None)
temp = temp.sort_values(by='date_taken')

# 17 mins for current operation

#To DO
#Implement logging to the whole operations

#Create view in Big Query
#Or create view initially. 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The length of the ID_list is 1064
The length of the ID_list is 1064


In [20]:
grade_levels = results[['assessment_id', 'grade_levels']].drop_duplicates()
temp = pd.merge(temp, grade_levels, on='assessment_id', how='left')

#Pull in curriculum, and Unit from other API endpoint merge up on assessment_id

In [25]:
temp

,assessment_id,title,local_student_id,first_name,last_name,middle_name,version,version_label,date_taken,points,points_possible,percent_correct,performance_band_level,performance_band_label,mastered,__count,Standard_No_Standard,last_update,academic_benchmark_guid,standard_code,standard_description,grade_levels
0,107650,1st Grade - IM Unit 8 - End of Unit Assessment,601773,Sophia,Vela,Alessandra,NaN,NaN,2000-01-01,4,4,100,4,Exceeds Expectations,t,1394,Standard,2024-10-21,A661A6CC-6F89-11DF-BAEE-EA329DFF4B22,CCSS.Math.Content.1.NBT.B.2,Understand that the two digits of a two-digit ...,1
1,107650,1st Grade - IM Unit 8 - End of Unit Assessment,601773,Sophia,Vela,Alessandra,NaN,NaN,2000-01-01,5,5,100,4,Exceeds Expectations,t,1394,Standard,2024-10-21,A66659B0-6F89-11DF-BAEE-EA329DFF4B22,CCSS.Math.Content.1.NBT.C.6,Subtract multiples of 10 in the range 10–90 fr...,1
2,107650,1st Grade - IM Unit 8 - End of Unit Assessment,601773,Sophia,Vela,Alessandra,NaN,NaN,2000-01-01,0,1,0,1,Does Not Meet Expectations,f,1394,Standard,2024-10-21,A667E604-6F89-11DF-BAEE-EA329DFF4B22,CCSS.Math.Content.1.MD.A.1,Order three objects by length; compare the len...,1
3,107650,1st Grade - IM Unit 8 - End of Unit Assessment,601773,Sophia,Vela,Alessandra,NaN,NaN,2000-01-01,3,3,100,4,Exceeds Expectations,t,1394,Standard,2024-10-21,A66356FC-6F89-11DF-BAEE-EA329DFF4B22,CCSS.Math.Content.1.NBT.B.2.b,The numbers from 11 to 19 are composed of a te...,1
4,107650,1st Grade - IM Unit 8 - End of Unit Assessment,601773,Sophia,Vela,Alessandra,NaN,NaN,2000-01-01,1,1,100,4,Exceeds Expectations,t,1394,Standard,2024-10-21,A6686BD8-6F89-11DF-BAEE-EA329DFF4B22,CCSS.Math.Content.1.MD.A.2,Express the length of an object as a whole num...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239948,107570,4th Grade- IM Unit 3- Section B Checkpoint,10542,Leah,Chavez,Mhyjoy,NaN,NaN,2024-10-21,0,4,0,1,Does Not Meet Expectations,f,406,Standard,2024-10-21,C51A26C2-96FF-11E0-9509-C03D9DFF4B22,4.MD.4,Make a line plot to display a data set of meas...,4
239949,107570,4th Grade- IM Unit 3- Section B Checkpoint,600219,Ezra,Catalan,Alexander,NaN,NaN,2024-10-21,0,4,0,1,Does Not Meet Expectations,f,406,Standard,2024-10-21,C51A26C2-96FF-11E0-9509-C03D9DFF4B22,4.MD.4,Make a line plot to display a data set of meas...,4
239950,107570,4th Grade- IM Unit 3- Section B Checkpoint,10156,Bria,Martin,Saryah,NaN,NaN,2024-10-21,1,9,11,1,Does Not Meet Expectations,f,406,Standard,2024-10-21,C50E17F6-96FF-11E0-9509-C03D9DFF4B22,4.NF.3,Understand a fraction a/b with a > 1 as a sum ...,4
239951,107622,1st Grade - IM Unit 2 - End of Unit Assessment,601730,Zion,Burr,Ryan,NaN,NaN,2024-10-21,1,1,100,4,Exceeds Expectations,t,1554,Standard,2024-10-21,A65A4396-6F89-11DF-BAEE-EA329DFF4B22,CCSS.Math.Content.1.OA.C.5,Relate counting to addition and subtraction (e...,1


In [26]:
academic_dashboard_version = temp[['date_taken', 'grade_levels', 'local_student_id', 'title', 'percent_correct', 'performance_band_level', 'performance_band_label']]

In [29]:
academic_dashboard_version

,date_taken,grade_levels,local_student_id,title,percent_correct,performance_band_level,performance_band_label
0,2000-01-01,1,601773,1st Grade - IM Unit 8 - End of Unit Assessment,100,4,Exceeds Expectations
1,2000-01-01,1,601773,1st Grade - IM Unit 8 - End of Unit Assessment,100,4,Exceeds Expectations
2,2000-01-01,1,601773,1st Grade - IM Unit 8 - End of Unit Assessment,0,1,Does Not Meet Expectations
3,2000-01-01,1,601773,1st Grade - IM Unit 8 - End of Unit Assessment,100,4,Exceeds Expectations
4,2000-01-01,1,601773,1st Grade - IM Unit 8 - End of Unit Assessment,100,4,Exceeds Expectations
...,...,...,...,...,...,...,...
239948,2024-10-21,4,10542,4th Grade- IM Unit 3- Section B Checkpoint,0,1,Does Not Meet Expectations
239949,2024-10-21,4,600219,4th Grade- IM Unit 3- Section B Checkpoint,0,1,Does Not Meet Expectations
239950,2024-10-21,4,10156,4th Grade- IM Unit 3- Section B Checkpoint,11,1,Does Not Meet Expectations
239951,2024-10-21,1,601730,1st Grade - IM Unit 2 - End of Unit Assessment,100,4,Exceeds Expectations


In [ ]:
#Takes 8 mins to run through 1064 right now on standard. 
#Take 9 mins to run through 1064 right now on no standard.

#These will expand out with time the more assessments that come through. 

